In [13]:
# !pip install swig
# !pip install gymnasium
# !pip install gymnasium[box2d]
# !pip install imageio

import numpy as np
import gymnasium as gym
import cv2
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from collections import namedtuple, deque


import imageio
import matplotlib.pyplot as plt
from IPython.display import Image, display
# For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
import glob
import base64, io
import matplotlib.animation as animation
from IPython.display import clear_output

In [167]:
def preprocess_image(img):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(),
        transforms.Resize((84, 84)),
        transforms.ToTensor()
    ])
    img = transform(img)
    return img


class ImageEnv(gym.Wrapper):
    def __init__(
            self,
            env,
            stack_frames=3,
            **kwargs
    ):
        super(ImageEnv, self).__init__(env, **kwargs)
        self.stack_frames = stack_frames
        self.skip_frames = 1
    def reset(self):
        # Reset the original environment.
        self.env.reset()

        s =preprocess_image(self.env.render())

        # The initial observation is simply a copy of the frame `s`
        self.stacked_state = s.repeat(self.stack_frames, 1, 1)
        return self.stacked_state

    def step(self, action):
        # We take an action for self.skip_frames steps
        reward = 0
        for _ in range(self.skip_frames):
            s, r, terminated, truncated, info = self.env.step(action)
            reward += r
            if terminated or truncated:
                break

        # Convert a frame to 84 X 84 gray scale one
        s =preprocess_image(self.env.render())

        # Push the current frame `s` at the end of self.stacked_state
        self.stacked_state=torch.cat((self.stacked_state[1:],s),dim=0)
        return self.stacked_state, reward, terminated, truncated, info

# Network

In [168]:
# 네트워크는 자유롭게 구성
# 아래는 예시

class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, actor_lr):
        super(PolicyNetwork, self).__init__()

        self.conv1=nn.Conv2d(state_dim[0],16,kernel_size=8,stride=4) #[N,4,84,84] -> [N,16,20,20]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=4, stride=2)  # [N, 16, 20, 20] -> [N, 32, 9, 9]
        self.in_features=32*9*9
        self.fc1=nn.Linear(self.in_features,256)
        self.fc_mu=nn.Linear(256,action_dim)
        self.fc_std=nn.Linear(256,action_dim)

        self.lr = actor_lr
        self.LOG_STD_MIN = -20
        self.LOG_STD_MAX = 2
        self.max_action = 2
        self.min_action = -2
        self.action_scale = (self.max_action - self.min_action) / 2.0
        self.action_bias = (self.max_action + self.min_action) / 2.0

        self.optimizer = optim.Adam(self.parameters(), lr=self.lr)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = x.view(-1, self.in_features)
        x = F.leaky_relu(self.fc1(x))
        mu = self.fc_mu(x)
        log_std = self.fc_std(x)
        log_std = torch.clamp(log_std, self.LOG_STD_MIN, self.LOG_STD_MAX)
        return mu, log_std

    def sample(self, state):
        mean, log_std = self.forward(state)
        std = torch.exp(log_std)
        reparameter = Normal(mean, std)
        x_t = reparameter.rsample()
        y_t = torch.tanh(x_t)
        action = self.action_scale * y_t + self.action_bias

        # # Enforcing Action Bound
        log_prob = reparameter.log_prob(x_t)
        log_prob = log_prob - torch.sum(torch.log(self.action_scale * (1 - y_t.pow(2)) + 1e-6), dim=-1, keepdim=True)

        return action, log_prob


class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, critic_lr):
        super(QNetwork, self).__init__()
        
        # First define convolutional layers to process the image state
        self.conv1 = nn.Conv2d(state_dim[0], 16, kernel_size=8, stride=4)  # [N, 3, 84, 84] -> [N, 16, 20, 20]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=4, stride=2)  # [N, 16, 20, 20] -> [N, 32, 9, 9]
        self.in_features = 32 * 9 * 9  # Flatten the output of the last conv layer to feed into a linear layer

        # Linear layers to process the flattened conv output
        self.fc1 = nn.Linear(self.in_features, 256)

        # Separate processing paths for state and action inputs
        self.fc_s = nn.Linear(256, 128)  # Process state features
        self.fc_a = nn.Linear(action_dim, 128)  # Process actions

        # Combine state and action processing paths
        self.fc_cat = nn.Linear(256, 128)  # Layer to combine state and action features
        self.fc_out = nn.Linear(128, 1)  # Output layer to produce a single Q-value

        # Learning rate and optimizer
        self.lr = critic_lr
        self.optimizer = optim.Adam(self.parameters(), lr=self.lr)

    def forward(self, x, a):
        # Process the image input through convolutional layers
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = x.view(-1, self.in_features)  # Flatten the output for the linear layer
        x = F.leaky_relu(self.fc1(x))

        # Further process state features
        s = F.leaky_relu(self.fc_s(x))
        
        # Process action features
        a = F.leaky_relu(self.fc_a(a))

        # Concatenate state and action features
        cat = torch.cat([s, a], dim=1)
        cat = F.leaky_relu(self.fc_cat(cat))

        # Output the Q-value
        q = self.fc_out(cat)
        return q

# Replay Buffer

In [169]:
class ReplayBuffer():
    def __init__(self, buffer_limit, DEVICE):
        self.buffer = deque(maxlen=buffer_limit)
        self.dev = DEVICE

    def update(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done = transition
            s_lst.append(s)
            a_lst.append(a)
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask = 0.0 if done else 1.0
            done_mask_lst.append([done_mask])

        s_array = np.array(s_lst, dtype=np.float32)
        a_array = np.array(a_lst, dtype=np.float32)
        r_array = np.array(r_lst, dtype=np.float32)
        s_prime_array = np.array(s_prime_lst, dtype=np.float32)
        done_array = np.array(done_mask_lst, dtype=np.float32)

        s_batch = torch.tensor(s_array, dtype=torch.float).to(self.dev)
        a_batch = torch.tensor(a_array, dtype=torch.float).to(self.dev)
        r_batch = torch.tensor(r_array, dtype=torch.float).to(self.dev)
        s_prime_batch = torch.tensor(s_prime_array, dtype=torch.float).to(self.dev)
        done_batch = torch.tensor(done_array, dtype=torch.float).to(self.dev)

        # r_batch = (r_batch - r_batch.mean()) / (r_batch.std() + 1e-7)

        return s_batch, a_batch, r_batch, s_prime_batch, done_batch

In [170]:
class SAC:
  """Implement"""
  def __init__(self, state_dim, action_dim):
      self.state_dim      = state_dim  # [cos(theta), sin(theta), theta_dot]
      self.action_dim     = action_dim  # [torque] in[-2,2]
      self.lr_pi          = 0.001
      self.lr_q           = 0.001
      self.gamma          = 0.98
      self.batch_size     = 200
      self.buffer_limit   = 100000
      self.tau            = 0.005   # for soft-update of Q using Q-target
      self.init_alpha     = 0.01
      self.target_entropy = -self.action_dim  # == -1
      self.lr_alpha       = 0.005
      self.DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      self.buffer         = ReplayBuffer(self.buffer_limit, self.DEVICE)
      print("사용 장치 : ", self.DEVICE)
      self.total_steps2 = 0
      self.warmup_steps = 5000
      self.log_alpha = torch.tensor(np.log(self.init_alpha)).to(self.DEVICE)
      self.log_alpha.requires_grad = True
      self.log_alpha_optimizer = optim.Adam([self.log_alpha], lr=self.lr_alpha)

      self.actor  = PolicyNetwork(self.state_dim, self.action_dim, self.lr_pi).to(self.DEVICE)
      self.Q1        = QNetwork(self.state_dim, self.action_dim, self.lr_q).to(self.DEVICE)
      self.Q1_target = QNetwork(self.state_dim, self.action_dim, self.lr_q).to(self.DEVICE)
      self.Q2        = QNetwork(self.state_dim, self.action_dim, self.lr_q).to(self.DEVICE)
      self.Q2_target = QNetwork(self.state_dim, self.action_dim, self.lr_q).to(self.DEVICE)

      self.Q1_target.load_state_dict(self.Q1.state_dict())
      self.Q2_target.load_state_dict(self.Q2.state_dict())

  def select_action(self, s):
      with torch.no_grad():
          action, log_prob = self.actor.sample(s.to(self.DEVICE))
      return action, log_prob

  def calc_target(self, mini_batch):
      s, a, r, s_prime, done = mini_batch
      with torch.no_grad():
          a_prime, log_prob_prime = self.actor.sample(s_prime)
          entropy = - self.log_alpha.exp() * log_prob_prime
          q1_target, q2_target = self.Q1_target(s_prime, a_prime), self.Q2_target(s_prime, a_prime)
          q_target = torch.min(q1_target, q2_target)
          target = r + self.gamma * done * (q_target + entropy)
      return target
  
  def process(self, transition):
    self.total_steps2 += 1
    self.buffer.update(transition)

    if self.total_steps2 > self.warmup_steps:
        self.train_agent()

    

  def train_agent(self):
      mini_batch = self.buffer.sample(self.batch_size)
      s_batch, a_batch, r_batch, s_prime_batch, done_batch = mini_batch

      td_target = self.calc_target(mini_batch)

      #### Q1 train ####
      q1_loss = F.smooth_l1_loss(self.Q1(s_batch, a_batch), td_target)
      self.Q1.optimizer.zero_grad()
      q1_loss.mean().backward()
      # nn.utils.clip_grad_norm_(self.q1.parameters(), 1.0)
      self.Q1.optimizer.step()
      #### Q1 train ####

      #### Q2 train ####
      q2_loss = F.smooth_l1_loss(self.Q2(s_batch, a_batch), td_target)
      self.Q2.optimizer.zero_grad()
      q2_loss.mean().backward()
      # nn.utils.clip_grad_norm_(self.q2.parameters(), 1.0)
      self.Q2.optimizer.step()
      #### Q2 train ####

      #### pi train ####
      a, log_prob = self.actor.sample(s_batch)
      entropy = -self.log_alpha.exp() * log_prob

      q1, q2 = self.Q1(s_batch, a), self.Q2(s_batch, a)
      q = torch.min(q1, q2)

      pi_loss = -(q + entropy)  # for gradient ascent
      self.actor.optimizer.zero_grad()
      pi_loss.mean().backward()
      # nn.utils.clip_grad_norm_(self.pi.parameters(), 2.0)
      self.actor.optimizer.step()
      #### pi train ####

      #### alpha train ####
      self.log_alpha_optimizer.zero_grad()
      alpha_loss = -(self.log_alpha.exp() * (log_prob + self.target_entropy).detach()).mean()
      alpha_loss.backward()
      self.log_alpha_optimizer.step()
      #### alpha train ####

      #### Q1, Q2 soft-update ####
      for param_target, param in zip(self.Q1_target.parameters(), self.Q1.parameters()):
          param_target.data.copy_(param_target.data * (1.0 - self.tau) + param.data * self.tau)
      for param_target, param in zip(self.Q2_target.parameters(), self.Q2.parameters()):
          param_target.data.copy_(param_target.data * (1.0 - self.tau) + param.data * self.tau)
      #### Q1, Q2 soft-update ####

# Evaluate

In [171]:
def evaluate(n_evals=1):
    eval_env = gym.make("Pendulum-v1", render_mode="rgb_array")
    eval_env = ImageEnv(eval_env)

    scores = 0
    for i in range(n_evals):
        s, done, ret = eval_env.reset(), False, 0
        while not done:
            a = agent.select_action(s)
            ns, r, done,truncated,_=eval_env.step(a)
            ret += r
            done = done or truncated
        scores += ret
    return np.round(scores / n_evals, 4)

# Training

In [172]:
# 아래 코드는 예시이며 수정 가능함
# 다만,pretrained weight을 저장하는 부분 (actor.pt)는 유지할 것

env = gym.make("Pendulum-v1",render_mode="rgb_array")
env=ImageEnv(env)

max_steps = int(1e6)
state_dim = (3, 84, 84)
action_dim = 1
agent = SAC(state_dim, action_dim)

history = {'Step': [], 'AvgReturn': []}
s=env.reset()
rewards=0
score=-1
total_steps=0

while True:
    wandb.init(project='SAC')
    wandb.run.name = 'SAC'
    wandb.run.save()

    if total_steps < 5000:
        a = env.action_space.sample()
    else:
        a, _ = agent.select_action(s)
        a = a.cpu().detach().numpy()
        a = a[0]
    ns, r, done,truncated,_ = env.step(a)
    total_steps+=1
    agent.process((s, a, r, ns, done))
    s = ns
    rewards+=r
    

    if done or truncated:
        s=env.reset()
        print("step:{} rewards: {}".format(total_steps,rewards))
        history['Step'].append(total_steps)
        history['rewards'].append(rewards)
        wandb.log({'Step': total_steps, 'AvgReturn': rewards})
        if score < rewards:
            score = rewards
            torch.save(agent.actor.state_dict(), 'actor.pt')
            print("New Best Saved")

        rewards=0

    if total_steps > max_steps:
        break

사용 장치 :  cpu
step:200 rewards: -940.1107795830923
step:400 rewards: -1112.7170185480127
step:600 rewards: -1396.5387809978297
step:800 rewards: -1412.9879868852315
step:1000 rewards: -1244.9905448269205
step:1200 rewards: -1309.4382248030486
step:1400 rewards: -1368.1377646152857
step:1600 rewards: -962.3256417863555
step:1800 rewards: -1353.5400821994103
step:2000 rewards: -1611.388806853789
step:2200 rewards: -1068.9012959524932
step:2400 rewards: -1452.6603223232337
step:2600 rewards: -1265.9375569674114
step:2800 rewards: -1437.393181602782
step:3000 rewards: -975.586931869438
step:3200 rewards: -1444.9547828726984
step:3400 rewards: -1259.5968925057473
step:3600 rewards: -873.3939511839313
step:3800 rewards: -915.2992095517907
step:4000 rewards: -1242.810752810679
step:4200 rewards: -1396.2622516635456
step:4400 rewards: -1405.0495817819208
step:4600 rewards: -969.043860664443
step:4800 rewards: -1069.0916007855199
step:5000 rewards: -1173.8044054166965


KeyboardInterrupt: 

# Visualize

In [ ]:
!mkdir video

state_dim = (3, 84, 84)
action_dim = 1
agent = SAC(state_dim, action_dim)

def show_video(env_name):
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = 'video/{}.mp4'.format(env_name)
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        return HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))
    else:
        print("Could not find video")

def save_video_of_model(policy, env_name):


    env = gym.make(env_name,render_mode="rgb_array")
    env=ImageEnv(env)
    agent.actor.load_state_dict(torch.load('/content/actor.pt'))
    s = env.reset()
    done = False
    frames=[]
    while not done:
        frame = env.render()
        frames.append(frame)
        a = agent.select_action(s)
        ns, r, done,truncated,_ = env.step(a)
        done = done or truncated
        s=ns

    env.close()
    imageio.mimsave("video/{}.mp4".format(env_name), frames, fps=30)

save_video_of_model(agent, 'Pendulum-v1')
show_video('Pendulum-v1')